In [1]:
import opensim as osim
import numpy as np
import hyperobjects as ho
import common
import clifford as cf
from clifford import g3c
import clifford.tools.g3c as g3c_tools
from opensim import Vec3
from enum import Enum

G3c, blades_g3c, stuff = g3c.layout, g3c.blades, g3c.stuff

e1, e2, e3 = g3c.e1, g3c.e2, g3c.e3
e12, e13, e23 = g3c.e12, g3c.e13, g3c.e23
e31 = -e13
Px, Py, Pz = g3c.e23, -g3c.e13, g3c.e12
up, down, homo = g3c.up, g3c.down, g3c.homo
no, ni = g3c.eo, g3c.einf
E0 = g3c.E0
I_base = g3c.I_base

c:\Users\glaisne\mambaforge\envs\workspace\Lib\site-packages\numba\core\decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)
c:\Users\glaisne\mambaforge\envs\workspace\Lib\site-packages\numba\core\decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)
c:\Users\glaisne\mambaforge\envs\workspace\Lib\site-packages\pyganja\__init__.py:2: UserWarning: Failed to import cef_gui, cef functions will be unavailable
  from .script_api import *
c:\Users\glaisne\mambaforge\envs\workspace\Lib\

In [2]:
class MultiVectorType(Enum):
    NotBlade = -1
    Vector = 0
    EuclideanPoint = 1
    ConformalPoint = 2
    PointPair = 3
    Circle = 4
    Line = 5
    Sphere = 6
    Plane = 7

def blade_type(mv):
    if not isinstance(mv, cf.MultiVector):
        return MultiVectorType.NotBlade
    
    obj = g3c_tools.interpret_multivector_as_object(mv)
    if obj == -1:
        return MultiVectorType.NotBlade
    elif obj == 0:
        return MultiVectorType.Vector
    elif obj == 1:
        return MultiVectorType.EuclideanPoint
    elif obj == 2:
        return MultiVectorType.ConformalPoint
    elif obj == 3:
        return MultiVectorType.PointPair
    elif obj == 4:
        return MultiVectorType.Circle
    elif obj == 5:
        return MultiVectorType.Line
    elif obj == 6:
        return MultiVectorType.Sphere
    elif obj == 7:
        return MultiVectorType.Plane
    
def to_vector(v):
    if isinstance(v, osim.Vec3):
        v = v.to_numpy()
    return v[0]*e1 + v[1]*e2 + v[2]*e3

def to_point(p):
    return up(to_vector(p))

In [9]:
n = e1 + e2 + e3
p = 4*e1
L = g3c_tools.random_line()
pla = n + (p | n)*ni
blade_type(L)
blade_type(L.dual())

<MultiVectorType.PointPair: 3>

In [3]:
def to_array(mv):
    b_type = blade_type(mv)
    if b_type == MultiVectorType.Vector:
        return np.array([mv | e1, mv | e2, mv | e3]).astype(float)
    elif b_type == MultiVectorType.EuclideanPoint:
        return np.array([mv | e1, mv | e2, mv | e3]).astype(float)
    elif b_type == MultiVectorType.ConformalPoint:
        mv = down(mv)
        return np.array([mv | e1, mv | e2, mv | e3]).astype(float)
    elif b_type == MultiVectorType.Line:
        # return position then direction
        P, D = g3c_tools.line_to_point_and_direction(mv)
        if P == 0:
            return [np.array([0,0,0]), to_array(D)]
        return [to_array(P), to_array(D)]
    elif b_type == MultiVectorType.Plane:
        # return orthogonal proj of origin onto it THEN normal
        location = g3c_tools.get_nearest_plane_point(mv)
        normal = g3c_tools.get_plane_normal(mv)
        if location == 0:
            return [np.array([0,0,0]), to_array(normal)]
        return [to_array(location), to_array(normal)]
    else:
        raise TypeError(f"Can't convert {b_type} as numpy array.")

def rotation_axis_in_joint_frame(model, Q, last_frame):
    W_in_ground = common.frame_jacobian(model, Q, last_frame)
    W_in_joints = []
    all_joints = [j for j in model.getJointSet()]
    for i, w_in_ground in enumerate(W_in_ground.T):
        Ji = all_joints[i // 3]
        w_in_joint = model.getGround().expressVectorInAnotherFrame(Q, Vec3(*w_in_ground), Ji.getParentFrame()).to_numpy()
        W_in_joints.append(w_in_joint)
    return np.array(W_in_joints).T

def lever_arms(model, Q, W, origin, frame_origin, insertion, frame_insertion):
    if blade_type(origin) == MultiVectorType.NotBlade:
        if not isinstance(origin, osim.Vec3):
            origin = osim.Vec3(*origin)
    else:
        origin = osim.Vec3(*to_array(origin))

    if blade_type(insertion) == MultiVectorType.NotBlade:
        if not isinstance(insertion, osim.Vec3):
            insertion = osim.Vec3(*insertion)
    else:
        insertion = osim.Vec3(*to_array(insertion))

    lever_arms = []
    for i, c in enumerate(model.getCoordinateSet()):
        # c: osim.Coordinate = c
        FJ: osim.Joint = c.getJoint().getParentFrame()
        O_in_joint = frame_origin.findStationLocationInAnotherFrame(Q, origin, FJ).to_numpy()
        I_in_joint = frame_insertion.findStationLocationInAnotherFrame(Q, insertion, FJ).to_numpy()
        moment = np.cross(I_in_joint, O_in_joint) / np.linalg.norm(I_in_joint - O_in_joint)
        lever_arm = W[:, i] @ moment
        lever_arms.append(lever_arm)
    
    return np.array(lever_arms)

In [44]:
J1_q1 = g3c_tools.random_rotation_translation_rotor()
J2_q1 = g3c_tools.random_rotation_translation_rotor()
J3_q1 = g3c_tools.random_rotation_translation_rotor()
J4_q1 = g3c_tools.random_rotation_translation_rotor()

J1_q2 = g3c_tools.random_rotation_translation_rotor()
J2_q2 = g3c_tools.random_rotation_translation_rotor()
J3_q2 = g3c_tools.random_rotation_translation_rotor()
J4_q2 = g3c_tools.random_rotation_translation_rotor()

J1_q3 = g3c_tools.random_rotation_translation_rotor()
J2_q3 = g3c_tools.random_rotation_translation_rotor()
J3_q3 = g3c_tools.random_rotation_translation_rotor()
J4_q3 = g3c_tools.random_rotation_translation_rotor()


O = g3c_tools.random_conformal_point() # in J4
I = g3c_tools.random_conformal_point() # in J2

O_in_ground_q1 = g3c_tools.apply_rotor(O, J4_q1 * J3_q1 * J2_q1 * J1_q1)
O_in_ground_q2 = g3c_tools.apply_rotor(O, J4_q2 * J3_q2 * J2_q2 * J1_q2)

I_in_ground_q1 = g3c_tools.apply_rotor(I, J1_q1)
I_in_ground_q2 = g3c_tools.apply_rotor(I, J1_q2)

L_q1 = O_in_ground_q1 ^ (I_in_ground_q1 - O_in_ground_q1) ^ ni
L_q2 = O_in_ground_q2 ^ (I_in_ground_q2 - O_in_ground_q2) ^ ni

# print(blade_type((J1_q1 * O * ~J1_q1) - (J2_q1 * O * ~J2_q2)))

K = J1_q1 - J2_q2
print(K)

0.52186 - (0.20742^e12) - (0.29226^e13) + (0.58905^e14) + (0.58905^e15) + (0.70852^e23) - (16.68462^e24) - (16.68462^e25) - (3.19775^e34) - (3.19775^e35) - (6.53129^e1234) - (6.53129^e1235)


In [14]:
(I_in_ground_q1 - O_in_ground_q1)

(38.59534^e1) + (5.64629^e2) + (54.75283^e3) + (179.69077^e4) + (179.69077^e5) - (0.0^e134) - (0.0^e135)

In [60]:

R1 = g3c_tools.random_rotation_rotor()
R2 = g3c_tools.random_rotation_rotor()


O = g3c_tools.random_conformal_point()
O = down(O)

print(((R1*O*~R1) / (R2*O*~R2)))
print((R1 / R2))

0.37369 - (0.3048^e12) - (0.65296^e13) + (0.58404^e23)
-0.79061 - (0.10759^e12) + (0.44064^e13) - (0.41134^e23)


In [69]:
print(blade_type(R1 << R2))
O = up(O)

MultiVectorType.NotBlade


In [73]:
print((R1 << R2) * O * ~(R1 << R2))
print((R1*O*~R1) << (R2*O* ~R2))

(5.48947^e1) + (6.22079^e2) - (2.03082^e3) + (36.36285^e4) + (37.35256^e5)
-46.64901
